# sfapi_client Demo


In [ ]:
from sfapi_client import Client
from sfapi_client.compute import Machine

from pathlib import Path
from authlib.jose import JsonWebKey
import json

# Exercise 1
## Setup keys and get user and project information
***

### I've stored my key is stored in a file in `~/.superfacility/`

In [ ]:
!ls -la ~/.superfacility/*.json

In [ ]:
# Paste the client_id for the key here
client_id = ""

# Get the path for your json file here
sfapi_key = Path().home() / ".superfacility" / "sfapi_training.json"

# This opens the json file and reads it into a format the client understands
with sfapi_key.open('r') as sfapi_json:
    client_secret = JsonWebKey.import_key(json.loads(sfapi_json.read()))
    
# We'll use `client_id` and `client_secret` through the rest of our tutorial

### Lets make sure we're authenticated and check who the api thinks we are

In [ ]:
# Create a client
client = Client(client_id, client_secret)

# Get the user info, "Who does the api think I am?"
user = client.user()

# Close the connection to the client
client.close()

# Let's see what the user object has in it
user

### All data from the object can be retrived from the property name

In [ ]:
# Get specific values from the `user` object
user.name

### Or serialized into a dict

In [ ]:
user.dict()

### Before we start let's make some useful variables for our home and scratch directory

#### Your home and scratch paths are based on your username 


* `/global/homes/username_first_letter/username`
* `/pscratch/sd/username_first_letter/username`


#### Let's make two variables to hold our scratch and home paths for later.

* __Bonus points for using the `user` object to automatically generate it__

In [ ]:
# Set your home directory 
home_path = # f"/global/homes/first_letter/username"

# Set your scratch directory
scratch_path = # f"/pscratch/sd/first_letter/username"

## Try it yourself!

1) Using the user object list the `projects` you are a part of.

2) From our list of projects print the hours given to the project.

3) From our list of projects print the hours left in the project.

4) From our list of projects print the disk usage of differnt groups in the project.

In [ ]:
with Client(client_id, client_secret) as client:
    user = client.user()
    # A user has projects associated with them
    # Check on your user projects
    # TODO: projects = ...
    # Projects will be a list of the project objects

In [ ]:
# For each project
# Print the information you want
for project in projects:
    ...
    
# It may be helpful to view what jupter displays for one project in the list
# Uncomment and run to see that the project object has in it
# projects[0]

***
# 
# 
# 
# 
# 
# Exercise 2
## Check Perlmutter status and queues
***

### Before we start any computing, let's check that Perlmutter is up.

Our compute object has information about the status of the machine.
It is also used to get the queue information and run commands and jobs on the system.

In [ ]:
with Client(client_id, client_secret) as client:
    perlmutter = client.compute(Machine.perlmutter)
    
perlmutter.status

### We can also check the status of other systems under the `resources` section

In [ ]:
with Client(client_id, client_secret) as client:
    nersc_status = client.resources.status()

# For each of the resources print the status
for name, status in nersc_status.items():
    print(f"{name: <20}| {status.description: <25}| {status.status}")

## Try it yourself

1) Use resources to any `outages` for Perlmutter.

2) Use the compute object to get the jobs in the partition `gpu_ss11`.

3) Which job is requesting the highest number of nodes right now?

    - If you're familiar with pandas: `pd.DataFrame([j.dict() for j in all_jobs])`
    
4) Use the compute object to get past jobs for a user.

#### Similar to getting the resourse status but now let's try outages

In [ ]:
with Client(client_id, client_secret) as client:
    # Similiar to before but this time let's get the outages
    # TODO: nersc_status = ...
    
# The `nersc_status` object is a dictionary of lists of outages
# Get the list you want from the dictionary based on the name
## nersc_status['perlmutter']

In [ ]:
# Find out the next time there's an scheduled outage
for outage in nersc_status['perlmutter']:
    ...

### Let's look at the `jobs` on Perlmutter

We can search just the gpu queue based on the slurm `partition` `'gpu_ss11'`

In [ ]:
with Client(client_id, client_secret) as client:
    perlmutter = client.compute(Machine.perlmutter)
    # TODO: all_jobs = ...

In [ ]:
# Loop through all the jobs and find the job with the highest number of nodes
max_nodes_job_id = None
max_nodes = 0

for job in all_jobs:
    # job.nodes is a string, remember to convert it to an int for the max comparison
    ...
        
print(max_nodes_job_id, max_nodes)

# For a bonus get the max_nodes_job_id and max_nodes for a job which `state` is "RUNNING"

### Jobs can also be easily loaded into pandas

In [ ]:
import pandas as pd
# Get the jobs into a dataframe
df = pd.DataFrame([j.dict() for j in all_jobs])
# Convert the column to ints
# ...
# Select jobs that are "RUNNING"
# ...
# Get the largest based on the id of the max and the location 
# ...

### We can also get previous jobs based on the jobid or the user who ran it

Let's see if you've run any jobs in the last ~24 hours (that's the default time frame `sacct` returns).

In [ ]:
with Client(client_id, client_secret) as client:
    perlmutter = client.compute(Machine.perlmutter)
    # TODO past_jobs =  ...

***
# 
# 
# 
# 
# 
# 
# Exercise 3
## Submit a job
***

## Now let's run a job

The job script below will run a simple python program to generate random numbers

In [ ]:
N = 1000

job_script = f"""#!/bin/bash

#SBATCH -q debug
#SBATCH -A ntrain3
#SBATCH -N 1
#SBATCH -C cpu
#SBATCH -t 00:10:00
#SBATCH -J sfapi-demo
#SBATCH --exclusive
#SBATCH --output={scratch_path}/sfapi-demo/sfapi-demo-%j.out
#SBATCH --error={scratch_path}/sfapi-demo/sfapi-demo-%j.error

module load python
# Prints N random numbers to form a normal disrobution
python -c "import numpy as np; numbers = np.random.normal(size={N}); [print(n) for n in numbers]"
""" 

#### Make sure your `scratch_path` is set properly in the job script and that it looks good to submit.

In [ ]:
print(job_script)

### Make sure our output folder is there for our data to go to

In [ ]:
with Client(client_id, client_secret) as client:
    perlmutter = client.compute(Machine.perlmutter)
    # This will run a command on perlmutter, here we use `mkdir` to make our output directory
    perlmutter.run(f"mkdir -p {scratch_path}/sfapi-demo")
    # We can run ls on the directory to see that it was created
    [output_dir] = perlmutter.ls(f"{scratch_path}/sfapi-demo", directory=True)

# Check that the directory is there
output_dir.is_dir()

## Try it yourself

1) Using our compute object submit a job to Perlmutter using the `job_script` and wait for it to complete.

2) Find out what node the job ran on.

2) Using our compute object `ls` to find the outputfile.

3) Download the output file and read it's contents.

4) Plot the results!

    - The output file has one number (as a string) per newline `\n`
    - Split the file by newline
    - Convert the numbers to floats, get rid of anything that's not a number
    - Plot the numbers in a histogram to see the distribution

### Submit the job and wait for the job to complete

In [ ]:
with Client(client_id, client_secret) as client:
    perlmutter = client.compute(Machine.perlmutter)
    # TODO job = ...
    # Let's save the job id to use later 
    # TODO job_id = ...
    # print(job_id)

    print(f"Waiting for job {job_id} to finish!")
    # Wait for the job to finish
    # TODO job...
    print("Done!")
    
    
job.state

### You can also get the same information about the job based on it's jobid

Use the `job_id` we saved before to print which node the job ran on.

In [ ]:
with Client(client_id, client_secret) as client:
    perlmutter = client.compute(Machine.perlmutter)
    job = perlmutter.job(jobid=job_id)

job.nodelist

### Read from the slurm output file

Our output file will name is based on the job id, "sfapi-demo-__jobid__.out" in the scratch directory we created before.

In [ ]:
with Client(client_id, client_secret) as client:
    perlmutter = client.compute(Machine.perlmutter)
    # TODO [output_file] = ...
    # Now let's download the file
    # TODO output_file_numbers = ...

### Read the output file and convert the strings to `float` 

In [ ]:
# TODO output_numbers = ...
# TODO numers = ...

In [ ]:
import matplotlib.pyplot as plt

# Plot a histogram to get the distribution

### Notebook by Nick Tyler

### Acknowledgements

- Bjoern Enders (Co-Leading the API Training)
- Chris Harris (Co-Author sfapi_client)
- Gabor Torok (Author api.nersc.gov)
- Charles Lively (Helping during training event)
- Seleste Rodriguez, Lipi Gupta (Helping with training accounts)